In [2]:
#This is a streamlined call that only fetches data for all Batteries mentioned in a list - actual generation in MW h, and aggregates each BM Unit by its zone with discharge and charge info separately
import requests
import pandas as pd
import os
from datetime import date

# --- Setup ---
start_date = date(2024, 6, 30)
end_date = date(2025, 7, 1)

# Load BMU to fuel type and GSP mapping
mapping_path = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data\BESS_Master_List.csv"
mapping_df = pd.read_csv(mapping_path)

# Ensure required columns exist
if 'NESO BMU ID' not in mapping_df.columns or 'GSP Zone' not in mapping_df.columns:
    raise ValueError("Mapping file must contain 'NESO BMU ID' and 'GSP Zone' columns.")

bm_units = mapping_df['NESO BMU ID'].dropna().unique().tolist()
if not bm_units:
    raise ValueError("No BM Units found in the mapping data.")

# --- API Call ---
base_url = "https://data.elexon.co.uk/bmrs/api/v1/datasets/B1610/stream"
params = {
    "from": start_date.isoformat(),
    "to": end_date.isoformat(),
    "bmUnit": bm_units
}

response = requests.get(base_url, params=params)
response.raise_for_status()
data = response.json()

df = pd.DataFrame(data)
if df.empty:
    raise ValueError("No data returned from B1610 API.")

# --- Merge & Processing ---
df_merged = df.merge(mapping_df, left_on="nationalGridBmUnitId", right_on="NESO BMU ID", how="left")

# Filter out rows without GSP Zone
df_merged = df_merged[~df_merged['GSP Zone'].isna()]

# Create positive and negative generation columns
df_merged['positive_generation'] = df_merged['quantity'].apply(lambda x: x if x > 0 else 0)
df_merged['negative_generation'] = df_merged['quantity'].apply(lambda x: x if x < 0 else 0)

# --- Aggregate at GSP Zone level ---
pivot_pos = df_merged.pivot_table(
    index=['settlementDate', 'settlementPeriod'],
    columns='GSP Zone',
    values='positive_generation',
    aggfunc='sum'
).add_prefix('pos_')

pivot_neg = df_merged.pivot_table(
    index=['settlementDate', 'settlementPeriod'],
    columns='GSP Zone',
    values='negative_generation',
    aggfunc='sum'
).add_prefix('neg_')

# Combine and order columns
pivot_df = pd.concat([pivot_pos, pivot_neg], axis=1)

# Order columns interleaved: pos_<zone>, neg_<zone>, ...
zones = sorted({col.split('_', 1)[1] for col in pivot_df.columns})
interleaved_cols = [f'pos_{zone}' for zone in zones if f'pos_{zone}' in pivot_df.columns] + \
                   [f'neg_{zone}' for zone in zones if f'neg_{zone}' in pivot_df.columns]

# Reorder
pivot_df = pivot_df.reset_index()
cols_order = ['settlementDate', 'settlementPeriod'] + interleaved_cols
pivot_df = pivot_df[cols_order]

# --- Save ---
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")
filename = f"GSPzoneGeneration_{start_str}_to_{end_str}.csv"

output_dir = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, filename)

pivot_df.to_csv(output_file, index=False)
print(f"Saved to {output_file}")


Saved to C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data\GSPzoneGeneration_2024-06-30_to_2025-07-01.csv
